### Target prop in a deep linear network

In [ ]:
%reset -f
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def rank_one_matrix():
    ''' return: random 2x2 rank one matrix'''
    W_ = np.random.randn(2,2)
    u,s,v = np.linalg.svd(W_)
    return np.outer(u[:,0],v[0])

def simple_matrix():
    ''' return: a specific 2x2 rank one matrix '''
    return np.outer(np.array([1,0]), np.array([1,2])/np.linalg.norm([1,2]))

def get_subspaces(A_, r):
    ''' return: a dict containing the four fundamental subspaces of a matrix A '''
    u,s,v = np.linalg.svd(A_, full_matrices=True)
    ss = {'im': u[:,:r], 'ker': v[r:].T, 'coim': v[:r].T, 'coker': u[:,r:]}
    return ss

def plot_vecline(V, i=0, color=0):
    ''' plots the 1d subspace (line) spanned by one vector V '''
    VV = np.concatenate((-10*V, 10*V), axis=1)
    ax[i].plot(VV[0], VV[1], color=sns.color_palette("RdBu_r",7)[color], linewidth=2.0)
    
def plot_quotient(V, W, i=0, color=5):
    ''' plots elements (lines) of the quotient space V/W 
        todo: replace n with sv or 1/sv
    '''
    for n in np.arange(-3,3.1,0.5):
        VV = np.concatenate((-10*V, 10*V), axis=1)
        VV = VV + n*W
        ax[i].plot(VV[0], VV[1], color=sns.color_palette("RdBu_r",7)[color], linewidth=0.5)

def square_axes(i=0):
    ''' make axis look nice '''
    fig.axes[i].axhline(0, color='w', linewidth=3.5)
    fig.axes[i].axvline(0, color='w', linewidth=3.5)
    fig.axes[i].set_xlim(-2,2)
    fig.axes[i].set_ylim(-2,2)
    fig.axes[i].set_aspect(1)
    fig.axes[i].get_xaxis().set_ticklabels([])
    fig.axes[i].get_yaxis().set_ticklabels([])

In [ ]:
# Forward activation
def forward(x_init, W_in):
    ''' get activations for all neurons '''
    # TODO: show two+ vectors / batch learning.
    # batch learning: first then second vs second then first vs batch -- commutivity? 
    x_ = []
    x_.append(x_init)
    for l in range(layers):
        x_.append(np.dot(W_in[l], x_[-1]))
    return x_

In [ ]:
# Backprop
def backward(x_end, W_in):
    ''' get dL/dx[l] for all x '''
    dL_ = (layers+1)*[None]
    dL_[-1] = (x_end-y) # negative gradient of MSE
    for l in range(layers-1,-1,-1):
        dL_[l] = np.dot(W_in[l].T, dL_[l+1])
    return dL_

def train_weights(W_in, dL_in, x_in, alpha=.1):
    ''' get dL/dW[l] for all l '''
    W_ = np.copy(W_in)
    for l in range(layers):
        W_[l] = W_[l] - alpha*np.outer(dL_in[l+1], x_in[l])
    return W_

def get_W_update(dL_t_in, x_t_in):
    ''' get np.outer(dL[l+1], x[l]), the rank-one update for the weights in backprop. '''
    W_up = train_steps*[layers*[None]]
    W_up_ss = train_steps*[layers*[None]]
    for i in range(train_steps):
        for l in range(layers):
            W_up[i][l] = np.outer(dL_t_in[i][l+1], x_t_in[i][l])
            W_up_ss[i][l] = get_subspaces(W_up[i][l], 1)
    return W_up, W_up_ss

In [ ]:
# Target prop
def get_pinv(W_in):
    ''' get pinv '''
    W_pinv_ = layers*[None]
    for l in range(layers):
        W_pinv_[l] = np.linalg.pinv(W_in[l])
    return W_pinv_

def get_targets(x_in, W_in, dL_in):
    '''  '''
    x_tar_ = (layers+1)*[None]
    W_pinv_ = get_pinv(W_in)
    x_tar_[-1] = x_in[-1] - alpha*dL_in[-1]
    for l in range(layers-1,-1,-1):
        x_tar_[l] = x_in[l] - np.dot(W_pinv_[l], x_in[l+1]) + np.dot(W_pinv_[l],x_tar_[l+1])
    return x_tar_

def train_weights_targ(W_in, x_targ_in, x_in, alpha=.1):
    ''' train weights, targ prop version '''
    dL_local = [x_in[l] - x_targ_in[l] for l in range(len(x_in))]
    W_ = train_weights(W_in, dL_local, x_in, alpha=alpha)
    return W_

In [ ]:
# construct model:
layers = 2
W = []
W_ss = []

train_steps = 0
alpha = .3

for l in range(layers):
    W.append(simple_matrix())
    W_ss.append(get_subspaces(W[l], 1))

# Data
x_0 = np.array([-1.5,-1.15])
y = np.array([1,0.5])
    
# Cost function visualization
vv, hh = np.mgrid[-2:2:20j, -2:2:20j]
L = 0.5*((hh-y[0])**2 + (vv-y[1])**2)

# First pass
x = forward(x_0, W)
dL = backward(x[-1], W)
x_tar = get_targets(x, W, dL)

# Training steps
x_bp_t = [x]
W_bp_t = [W]
dL_t = [dL]

x_tp_t = [x]
x_tar_t = [x_tar]
W_tp_t = [W]

# Training
for i in range(train_steps):
    # backprop
    W_bp_t.append(train_weights(W_bp_t[-1], dL_t[-1], x_bp_t[-1], alpha=alpha))
    x_bp_t.append(forward(x_0, W_bp_t[i]))
    dL_t.append(backward(x_bp_t[-1][-1], W_bp_t[-1]))
    
    # target prop
    W_tp_t.append(train_weights_targ(W_tp_t[-1], x_tar_t[-1], x_tp_t[-1], alpha=alpha))
    x_tp_t.append(forward(x_0, W_tp_t[i]))
    x_tar_t.append(get_targets(x_tp_t[-1], W_tp_t[-1], dL_t[-1]))

# pseudoinv
W_pi = []
W_ps = []
for i in range(train_steps+1):
    W_pi.append(get_pinv(W_tp_t[i]))
for l in range(layers):
    W_ps.append(get_subspaces(W_pi[0][l], 1))

In [ ]:
def plot_dots(ax_):
    for l in range(layers+1):
        for t in [c+1 for c in range(train_steps-2,-1,-1)]:
            #ax_[l].plot(x_bp_t[t+1][l][0], x_bp_t[t+1][l][1], 'o', color=sns.color_palette()[0], ms=10, alpha=0.1)
            #ax_[l].plot(dL_t[t+1][l][0], dL_t[t+1][l][1], 'o', color=sns.color_palette()[1], ms=10, alpha=0.1)
            ax_[l].plot(x_tp_t[t+1][l][0], x_tp_t[t+1][l][1], '*', color=sns.color_palette()[0], ms=10, alpha=0.5)
            ax_[l].plot(x_tar_t[t+1][l][0], x_tar_t[t+1][l][1], '*', color=sns.color_palette()[1], ms=10, alpha=0.5)

        #ax_[l].plot(x_bp_t[0][l][0], x_bp_t[0][l][1], 'o', color=sns.color_palette()[0], ms=10)
        #ax_[l].plot(dL_t[0][l][0], dL_t[0][l][1], 'o', color=sns.color_palette()[1], ms=10)
        ax_[l].plot(x_tp_t[0][l][0], x_tp_t[0][l][1], '*', color=sns.color_palette()[0], ms=10)
        ax_[l].plot(x_tar_t[0][l][0], x_tar_t[0][l][1], '*', color=sns.color_palette()[1], ms=10)
    ax_[-1].plot(y[0],y[1], '^', color=sns.color_palette()[4], ms=10)


In [ ]:
## ROW 1
fig, ax = plt.subplots(1, layers+1, figsize=(20,20))
for l in range(layers+1):
    square_axes(l)

for l in range(layers):
    plot_quotient(W_ss[l]['ker'], W_ss[l]['coim'], l)
    plot_vecline(W_ss[l]['ker'], l, -1)
    plot_vecline(W_ss[l]['coim'],l, 1)

plot_dots(ax)
ax[-1].contour(hh, vv, L, 10)
plt.show()

## ROW 2
fig, ax = plt.subplots(1, layers+1, figsize=(20,20))
for l in range(layers+1):
    square_axes(l)

for l in range(layers):
    plot_quotient(W_ss[l]['coker'], W_ss[l]['im'], l+1)
    plot_vecline(W_ss[l]['coker'], l+1, -1)
    plot_vecline(W_ss[l]['im'],l+1, 1)

plot_dots(ax)
plt.show()

- W[l] moves in the direction to minimize error.
- Does this imply that x[l] also moves in the direction to minimize error?


- Specify the space of possible solutions -- global sections of x[l], but also space of W[l]?


- visualize the gradient functionals 


- Understand why for backprop:
    - dL/dW does not change over training steps (uh, maybe a bug?)
    - dL/dW is specifically rank-one. What is gained by higher-rank updates. Relation to FORCE / RLS / Full-FORCE. Newton's method -> higher thank rank-one?
    

- should be able to prove why W_update is constant for deep linear networks. Implications?


- confirm: does x[t] + alpha*dL/dx[t] = x[t] + alpha*(x_prevlayer[t])

dL/dx[l] says: move in this direction to reduce error. And dL/dW[l] says: 


- idea: plot loss contours at each layer -- i.e. backprop the entire error function... (?)


- more generally: play with visualizations for images, kernel, preimage, etc.

In [ ]:
x_array = np.array(x_tp_t)
dL_array = np.array(dL_t)
W_array = np.array(W_tp_t)
#W_update_array = np.array(W_update)

In [ ]:
fig, ax = plt.subplots(1, layers+1, figsize=(20,4))
for l in range(layers+1):
    ax[l].plot(x_array[:,l,:])
    ax[l].plot(dL_array[:,l,:],'--')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, layers, figsize=(20,4))
for l in range(layers):
    ax[l].plot(W_array[:,l,:,0])
    ax[l].plot(W_array[:,l,:,1])
    ax[l].plot(W_update_array[:,l,:,0],'--')
    ax[l].plot(W_update_array[:,l,:,1],'--')
plt.show()

In [ ]:
W_update_array.shape

In [ ]:
tr = 30
print x_t[tr][2]
print y
print x_t[tr][2] - y
print x_t[tr][1]

print np.outer(x_t[tr][2]-y, x_t[tr][1])


### Target prop in tensorflow

In [1]:
%reset -f
%matplotlib inline

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns

In [3]:
run = 0

In [4]:
### Toy data.
vv, hh = np.mgrid[-2:2:50j, -2:2:50j]

x_data = np.stack((vv.ravel(), hh.ravel()), axis=1)

# y data
rad = np.linalg.norm(x_data, axis=1)
y_data = 2*np.logical_and(rad>0.8, rad<1.2).astype('float')-1
y_data = np.stack((y_data, np.zeros(y_data.shape)), axis=1)

# params
m_dim = 2
p_dim = 2
batch_size = 100
num_samples = y_data.shape[0]

In [5]:
def get_rand_batch(data):
    inds = np.random.choice(num_samples, batch_size)
    for i in range(len(data)):
        data[i] = data[i][inds]
    return inds, data

In [6]:
### Mnist data
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

mnist_data = input_data.read_data_sets("MNIST_data/", one_hot=True)
num_samples = mnist_data.train.num_examples
m_dim = mnist.IMAGE_PIXELS
p_dim = mnist.NUM_CLASSES
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
x_data = mnist_data.train.images
y_data = mnist_data.train.labels

In [28]:
### Model
tf.reset_default_graph()

layers = 5
l_dim = [m_dim] + (layers-1)*[100] + [p_dim]
stddev = 0.01
b_init = 0.1

alpha = 1

# Placeholders
x_in = tf.placeholder(tf.float32, shape=[batch_size, m_dim], name='x0')
y = tf.placeholder(tf.float32, shape=[batch_size, p_dim], name='y')
epoch = tf.placeholder(tf.float32, shape=None, name='std')

noise_inj = 10/(1+epoch/100)

# Initialize lists
b = (layers+1)*[None]
W = (layers+1)*[None]
x = (layers+1)*[None]
x[0] = x_in

x_tar = (layers+1)*[None]
V = (layers+1)*[None]
c = (layers+1)*[None]

gx = (layers+1)*[None]
gx_tar = (layers+1)*[None]

L = (layers+1)*[None]
L_inv = (layers+1)*[None]

# Forward graph
for l in range(1, layers+1):
    with tf.name_scope('layer'+str(l)) as scope:
        b[l] = tf.Variable(tf.constant(b_init, shape=[1, l_dim[l]]), name='b_'+str(l))
        W[l] = tf.Variable(tf.truncated_normal([l_dim[l-1], l_dim[l]], stddev=stddev), name='W_'+str(l))
        x[l] = tf.add(tf.matmul(x[l-1], W[l]), b[l], name='x_'+str(l)) # better way to name?

# Global loss
#L_g = tf.reduce_mean(0.5*(x[-1] - y)**2, name="global_loss")
L_g = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.nn.softmax(x[-1])), reduction_indices=[1]), name="global_loss")

# Top-layer targets
x_tar[-1] = x[-1] - alpha*tf.gradients(L_g, x[-1])[0]
x_tar[-2] = x[-2] - alpha*tf.gradients(L_g, x[-2])[0] # tf.gradients can't go backward through the graph... 

# Target graph
for l in range(layers,1,-1): # from M to 2
    with tf.name_scope('layer_target'+str(l)) as scope:
        c[l] = tf.Variable(tf.constant(b_init, shape=[1, l_dim[l-1]]), name='c_'+str(l))
        #c[l] = tf.Variable(tf.constant(b_init, shape=[1, l_dim[l-1]]), name='c_'+str(l))
        V[l] = tf.Variable(tf.truncated_normal([l_dim[l], l_dim[l-1]], stddev=stddev), name='V_'+str(l))
        gx[l] = tf.nn.sigmoid(tf.matmul(x[l], V[l]) + c[l], name='g_x_'+str(l))
        gx_tar[l] = tf.nn.sigmoid(tf.matmul(x_tar[l], V[l]) + c[l], name='g_x_tar'+str(l))
        if x_tar[l-1] is None:
            x_tar[l-1] = x[l-1] + gx_tar[l] - gx[l] # give name
        

TensorShape([Dimension(1), Dimension(10)])

In [30]:
# Optimizers
opt = tf.train.GradientDescentOptimizer(0.5)

train_op_inv = (layers+1)*[None]
train_op = (layers+1)*[None]
    
fx_ = (layers+1)*[None]
gx_ = (layers+1)*[None]

for l in range(1, layers+1):
    with tf.name_scope('layer'+str(l)) as scope:
        L[l] = tf.reduce_mean(0.5*(x[l] - x_tar[l])**2, name="L"+str(l))
        train_op[l] = opt.minimize(L[l], var_list=[W[l], b[l]])

for l in range(2, layers+1):
    with tf.name_scope('layer'+str(l)) as scope:
        fx_[l] = tf.nn.sigmoid(tf.matmul(x[l-1], W[l]) + b[l] + tf.random_normal(b[l].get_shape(), stddev=noise_inj))
        gx_[l] = tf.nn.sigmoid(tf.matmul(fx_[l], V[l]) + c[l])
        L_inv[l] = tf.reduce_mean(0.5*(gx_[l] - (x[l-1] + tf.random_normal(c[l].get_shape(), stddev=noise_inj)))**2,
                                  name='L_inv_'+str(l))
        train_op_inv[l] = opt.minimize(L_inv[l], var_list=[V[l], c[l]])

train_bp = opt.minimize(L_g, var_list=[i for i in W+b if i is not None])

correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(x[-1]), 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
# clean up
train_op = [i for i in train_op if i is not None]
train_op_inv = [i for i in train_op_inv if i is not None]

In [32]:
for l in range(layers):
    if L[l] is not None:
        tf.scalar_summary('L'+str(l), L[l])
    if L_inv[l] is not None:
        tf.scalar_summary('L_inv'+str(l), L_inv[l])
tf.scalar_summary('L_g', L_g)

#tf.scalar_summary('learning rate', train_op._lr_t)
#tf.scalar_summary('learning rate2', train_op._lr)
#tf.scalar_summary('accuracy', accuracy)

merged_summary_op = tf.merge_all_summaries()

In [33]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

run+=1

summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs/'+str(run), sess.graph)

for i in range(10000):
    inds_batch, [x_batch, y_batch] = get_rand_batch([x_data, y_data])
    feed_dict={x_in: x_batch, y: y_batch, epoch: i}
    # first train Linv
    #sess.run(train_op, feed_dict=feed_dict)
    #sess.run(train_op_inv, feed_dict=feed_dict)
    sess.run(train_bp, feed_dict=feed_dict)
    loss_val, summary_str, acc_val = sess.run([L_g, merged_summary_op, accuracy], feed_dict=feed_dict)
    summary_writer.add_summary(summary_str, i)
    
    if i % 100 == 0:
        print "iter:", "%04d" % (i), \
              "loss:", "{:.4f}".format(loss_val), \
              "accuracy:", "{:.4f}".format(acc_val)

print "finished"
summary_writer.close()

iter: 0000 loss: 2.3013 accuracy: 0.1300
iter: 0100 loss: 2.2949 accuracy: 0.1500
iter: 0200 loss: 2.2973 accuracy: 0.0800
iter: 0300 loss: 2.2902 accuracy: 0.1300
iter: 0400 loss: 2.3027 accuracy: 0.1300
iter: 0500 loss: 2.3001 accuracy: 0.1100
iter: 0600 loss: 2.2909 accuracy: 0.1500
iter: 0700 loss: 2.2886 accuracy: 0.1700
iter: 0800 loss: 2.3010 accuracy: 0.1000
iter: 0900 loss: 2.3045 accuracy: 0.1100
iter: 1000 loss: 2.3044 accuracy: 0.1100
iter: 1100 loss: 2.3027 accuracy: 0.1000
iter: 1200 loss: 2.3024 accuracy: 0.1200
iter: 1300 loss: 2.2952 accuracy: 0.1400
iter: 1400 loss: 2.3022 accuracy: 0.1000


KeyboardInterrupt: 